# Joe's cool code

In [1]:
# Import General Libs
import pandas as pd
import os
import scipy as sp
import numpy as np

# Import project libs
import Data.DataWrangle as DW

In [4]:
#Data setup
cleanData = DW.read_into_MC_format(with_pct_change=True)
cleanData

SPY                TR               VIX           
             close pct_change  close pct_change  close pct_change
timestamp                                                        
1990-01-02     NaN        NaN    NaN        NaN  17.24        NaN
1990-01-03     NaN        NaN    NaN        NaN  18.19   0.055104
1990-01-04     NaN        NaN    NaN        NaN  19.22   0.056625
1990-01-05     NaN        NaN    NaN        NaN  20.11   0.046306
1990-01-08     NaN        NaN    NaN        NaN  20.26   0.007459
...            ...        ...    ...        ...    ...        ...
2022-03-24  450.48   0.015052  6.700   0.139456  21.67  -0.080611
2022-03-25  452.69   0.004906  4.550  -0.320896  20.81  -0.039686
2022-03-28  455.91   0.007113  5.850   0.285714  19.63  -0.056704
2022-03-29  461.55   0.012371  6.160   0.052991  18.90  -0.037188
2022-03-30  458.70  -0.006175  5.085  -0.174513    NaN        NaN

[8121 rows x 6 columns]

In [3]:
def add_rolling_integrate(df:pd.DataFrame,data_col,rollNum,):
    data = df[data_col]
    data.rolling(rollNum).apply(lambda x : sp.integrate.trapz)